In [9]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
import openvino as ov
import matplotlib.pyplot as plt
import os

# Reference the below URL for additional guideance if needed
# https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/openvino-api/openvino-api.ipynb

In [10]:
# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import download_file

In [11]:
# This Chunk downloads the model if it needs to be downloaded

base_artifacts_dir = Path('./artifacts').expanduser()

model_name = "face-detection-0200"
model_xml_name = f'{model_name}.xml'
model_bin_name = f'{model_name}.bin'

model_xml_path = base_artifacts_dir / model_xml_name

base_url = 'https://storage.openvinotoolkit.org/repositories/open_model_zoo/temp/face-detection-0200/FP32/'

if not model_xml_path.exists():
    download_file(base_url + model_xml_name, model_xml_name, base_artifacts_dir)
    download_file(base_url + model_bin_name, model_bin_name, base_artifacts_dir)
else:
    print(f'{model_name} already downloaded to {base_artifacts_dir}')

face-detection-0200 already downloaded to artifacts


In [12]:
# This chunk specifies what device to run the model on i.e. CPU vs GPU
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

In [13]:
# This chunk instaniates the model with the open vino objects

core = ov.Core()
model = core.read_model(model=model_xml_path)
compiled_model = core.compile_model(model=model, device_name=device.value)

output_layer = compiled_model.output(0)

In [14]:
input_layer = compiled_model.input(0)

print(f"input precision: {input_layer.element_type}")
print(f"input shape: {input_layer.shape}")
print(f"output precision: {output_layer.element_type}")
print(f"output shape: {output_layer.shape}")

input precision: <Type: 'float32'>
input shape: [1,3,256,256]
output precision: <Type: 'float32'>
output shape: [1,1,200,7]


The net outputs blob with shape: 1, 1, 200, 7 in the format 1, 1, N, 7, where N is the number of detected bounding boxes. Each detection has the format [image_id, label, conf, x_min, y_min, x_max, y_max], where:

    image_id - ID of the image in the batch
    label - predicted class ID (0 - face)
    conf - confidence for the predicted class
    (x_min, y_min) - coordinates of the top left bounding box corner
    (x_max, y_max) - coordinates of the bottom right bounding box corner


In [17]:
# Create an empty list to store image metadata
image_meta = []

# Counter to limit to the first 5 files
#file_count = 0

image_files = os.listdir('train')

# Loop through images in the folder
for filename in tqdm(image_files, desc="Processing images"):
    
    # Stop after processing 5 files
    #if file_count >= 10:
        #break

    # Get the file path
    image_path = os.path.join('train', filename)

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read {image_path}")
        continue

    # Increment the file count after a successful read
    #file_count += 1

    # Store the original dimensions
    orig_h, orig_w = image.shape[:2]

    # Resize to (256, 256) and prepare the input
    resized_image = cv2.resize(image, (256, 256))

    # Reorder dimensions to (C, H, W) from (H, W, C)
    input_image = np.transpose(resized_image, (2, 0, 1))

    # Add batch dimension (B=1)
    input_image = np.expand_dims(input_image, axis=0)

    # Run the model
    outputs = compiled_model([input_image])[output_layer]

    # Process the output (assuming shape (1, 1, N, 7))
    detections = np.squeeze(outputs, axis=(0, 1))
    
    # Create dummy holding 10392dictionary
    image_info = {"file_path": image_path, "confidence": 0}

    # Grab only the first bounding box if available
    if len(detections) > 0:
        detection = detections[0]
        conf = float(detection[2])
        if conf > 0.01:  # Confidence threshold
            # Bounding box coordinates (relative to the original image size)
            x_min = int(detection[3] * orig_w)
            y_min = int(detection[4] * orig_h)
            x_max = int(detection[5] * orig_w)
            y_max = int(detection[6] * orig_h)

            # Draw the bounding box on the original image
            #cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            #cv2.putText(image, f"Conf: {conf:.2f}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Update the dictionary with confidence
            image_info["confidence"] = conf

    # Append the confidence and file path to the metadata list
    image_meta.append(image_info)

    # Display the result using Matplotlib
    #image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #plt.imshow(image_rgb)
    #plt.title(f"Face Detection - {filename}")
    #plt.axis('off')
    #plt.show()

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(image_meta)

# Optionally, you can save it to a CSV if needed
results_df.to_csv("image_metadata.csv", index=False)
print("Processed images and stored metadata.")

Processing images: 100%|██████████| 86744/86744 [11:06<00:00, 130.18it/s]


Processed images and stored metadata.


In [16]:
results_df.head

<bound method NDFrame.head of              file_path  confidence
0      train/64277.jpg    0.997335
1      train/13135.jpg    0.900043
2      train/47394.jpg    0.890959
3      train/50091.jpg    0.996324
4      train/26594.jpg    0.997323
...                ...         ...
86739  train/81284.jpg    0.999067
86740  train/78930.jpg    0.992692
86741  train/31351.jpg    0.997285
86742  train/67347.jpg    0.921577
86743  train/64566.jpg    0.925131

[86744 rows x 2 columns]>